### Build MemSQL Connection

In [1]:
import pymysql
import pandas 

HOST = "127.0.0.1"
PORT = 3306
USER = "root"
PASSWORD = ""
DATABASE = "nba"

conn = pymysql.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=DATABASE, charset='utf8mb4')

### Fetch Date, Teams, Scores and Broadcasters for Given Season

In [2]:
def get_all_2017_18_games():
    """ Create a database and table for this benchmark to use. """

    return pandas.read_sql_query(""" 
        SELECT
            gh.game_date,
            gh.natl_tv_broadcaster,
            ht.name home_team_name,
            at.name away_team_name,
            hls.pts home_team_pts,
            als.pts away_team_pts
        FROM
            game_header gh
            
        -- Get the home team information
        JOIN team ht ON
            gh.home_team_id = ht.id
        JOIN line_score hls ON
            hls.game_id = gh.game_id AND
            hls.team_id = ht.id

        -- Get the visiting team information
        JOIN team at ON
            gh.away_team_id = at.id
        JOIN line_score als ON
            als.game_id = gh.game_id AND
            als.team_id = at.id
            
        ORDER BY gh.game_date DESC
    """, conn)

games_df = get_all_2017_18_games()

In [2]:
import plotly.plotly as ply
from plotly.graph_objs import *

game_names = []
for i, gdf in games_df.iterrows():
    game_name = '[%s] %s (%s) @ %s (%s)' % (
        gdf['game_date'], 
        gdf['home_team_name'], 
        gdf['home_team_pts'],
        gdf['away_team_name'], 
        gdf['away_team_pts']
    )
    game_names.append(game_name)

trace1 = Scatter(
     x=games_df['home_team_pts'],
     y=games_df['away_team_pts'],
     text=game_names,
     mode='markers'
)

layout = Layout(
     xaxis=XAxis( title='Home Team Points' ),
     yaxis=YAxis( type='log', title='Visitor Team Points' )
)

data = Data([trace1])
fig = Figure(data=data, layout=layout)
ply.iplot(fig, filename='Home Team Points v Away Team Points Comparison')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~NeilDahlke/0 or inside your plot.ly account where it is named 'Home Team Points v Away Team Points Comparison'


### Heatmap of point differentials for each team.

In [3]:
import plotly.plotly as ply
from plotly.graph_objs import *

heatmap_matrix = []
teams = {}

for i, gdf in games_df.iterrows():
    home_name = gdf['home_team_name']
    away_name = gdf['away_team_name']
    home_pts = gdf['home_team_pts']
    away_pts = gdf['away_team_pts']
    
    if home_name not in teams:
        teams[home_name] = {}
    
    if away_name not in teams:
        teams[away_name] = {}
        
    if away_name not in teams[home_name]:
        teams[home_name][away_name] = 0

    if home_name not in teams[away_name]:
        teams[away_name][home_name] = 0

    
    teams[away_name][home_name] += home_pts
    teams[home_name][away_name] += away_pts
    """
    if (away_name == "Warriors" or home_name == "Warriors") and \
        (away_name == "Cavaliers" or home_name == "Cavaliers"):
        print(away_name, away_pts, '@', home_name, home_pts)
    """

sorted_team_names = sorted(teams.keys())

layout = Layout(
     xaxis=XAxis(title='Home Team Name'),
     yaxis=YAxis(title='Away Team Name')
)

point_differentials = []
for x_team in sorted_team_names:
    z = []
    for y_team in sorted_team_names:
        if x_team == y_team:
            z.append(0)
        else:
            delta = teams[x_team][y_team] - teams[y_team][x_team]
            z.append(delta)
    point_differentials.append(z)

trace = Heatmap(
    x=sorted_team_names,
    y=sorted_team_names,
    z=point_differentials,
    colorscale='Blackbody'
)
data=[trace]
ply.iplot(data, filename='basic-heatmap')